Beispiel mit unshare

In [ ]:
! pstree -npTA

In [ ]:
! ip addr

In [ ]:
! cat /etc/issue

Wir holen uns ein Alpine Linux und führen ein paar Befehle einem eigenen Linux Namespace aus

In [ ]:
%%bash
wget -qqq -O alpine.tar.gz http://dl-cdn.alpinelinux.org/alpine/latest-stable/releases/x86_64/alpine-minirootfs-3.20.1-x86_64.tar.gz
rm -rf alpine && mkdir -p alpine
tar -C alpine -xvf alpine.tar.gz
rm alpine.tar.gz
# Netzwerk
sudo cp /etc/resolv.conf alpine/etc/
# zusätzliche SW installieren
sudo unshare -p --fork --mount-proc --root alpine apk update 
sudo unshare -p --fork --mount-proc --root alpine apk add pstree iproute2 iputils iptables

In [ ]:
# Filesystem
! sudo unshare -p --fork --mount-proc --root alpine cat /etc/issue

In [ ]:
# Prozesse
! sudo unshare -p --fork --mount-proc --root alpine pstree

In [ ]:
# Netzwerk
! sudo unshare -p --fork --mount-proc --net --root alpine ip addr

Nun könnten wir noch einen Netzwerkadapter im Alpine Linux Namespace erstellen.

Die Befehle sind etwa so

    # Erstellen eines veth-Paares
    sudo ip link add veth0 type veth peer name veth1
    
    # Starten des unshare-Prozesses in einer neuen Shell und Ermitteln der PID
    sudo unshare -p --fork --mount-proc --root alpine sh -c "
      sleep 120
      ip addr add 192.168.100.2/24 dev veth1
      ip link set veth1 up
      ip route add default via 192.168.100.1
    " &
    UNSHARE_PID=$!
    
    # Warten, bis der unshare-Prozess startet
    sleep 1
    
    # Verschieben von veth1 in den neuen Netzwerk-Namespace
    sudo ip link set veth1 netns $UNSHARE_PID
    
    # Konfiguration von veth0 auf dem Host
    sudo ip addr add 192.168.200.1/24 dev veth0
    sudo ip link set veth0 up
    
    # IP-Weiterleitung auf dem Host aktivieren
    echo 1 | sudo tee /proc/sys/net/ipv4/ip_forward
    
    # Konfiguration von NAT auf dem Host
    sudo iptables -t nat -A POSTROUTING -s 192.168.200.0/24 -o eth0 -j MASQUERADE
    
    # Warten Sie, bis der unshare-Prozess beendet ist (falls erforderlich)
    wait $UNSHARE_PID


